# ANÁLISIS SOBRE METODOS

In [ ]:
#Imports
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.model_selection import train_test_split

: 

# Selección de métricas

In [ ]:
# Creación del conjunto de datos de entreno y de test
train_dataset = pd.read_csv(("./datasets/method.csv"), index_col = 'LongName')
train_dataset, test_dataset = train_test_split(train_dataset, test_size=0.2)

display(train_dataset.head(3))
display(test_dataset.head(3))

In [ ]:
# Nombre de la columna donde se encuentra el resultado del fichero
salida = 'Number of Bugs'
# Columnas del dataset 
features = ["CC","CCL","CCO","CI","CLC","CLLC","LDC","LLDC","HCPL","HDIF","HEFF","HNDB","HPL","HPV","HTRP","HVOL","MI","MIMS","MISEI","MISM","McCC","NL","NLE","NII","NOI","CD","CLOC","DLOC","TCD","TCLOC","LLOC","LOC","NOS","NUMPAR","TLLOC","TLOC","TNOS","WarningBlocker","WarningCritical","WarningInfo","WarningMajor","WarningMinor","Android Rules","Basic Rules","Brace Rules","Clone Implementation Rules","Code Size Rules","Comment Rules","Controversial Rules","Coupling Rules","Design Rules","Empty Code Rules","Finalizer Rules","Import Statement Rules","J2EE Rules","JUnit Rules","Jakarta Commons Logging Rules","Java Logging Rules","JavaBean Rules","MigratingToJUnit4 Rules","Migration Rules","Migration13 Rules","Migration14 Rules","Migration15 Rules","Naming Rules","Optimization Rules","Security Code Guideline Rules","Strict Exception Rules","String and StringBuffer Rules","Type Resolution Rules","Unnecessary and Unused Code Rules","Vulnerability Rules"]

# TODO En la columna de outcome se indica la cantidad de bugs que hay en ese metodo, hay que gestionar este outcome para que consiga interpretar un valor no binario (0, 1) si no tambien la cantidad 0-6 
# Creacion de una nueva columna con valores (0, 1)
# Se indica que la feature outcome es una variable categórica, que toma valores en un intervalo limitado
train_dataset[salida] = pd.Categorical(train_dataset[salida])
display(train_dataset[salida])
# Convertimos las variables en códigos en función de la categoría 
train_dataset[salida] = train_dataset[salida].cat.codes
display(train_dataset[salida])


In [ ]:
# Hacemos la misma conversión con los datos del conjunto de test
test_dataset[salida] = pd.Categorical(test_dataset[salida])
test_dataset[salida] = test_dataset[salida].cat.codes

# Se construyen los dataset con las features, separándolos de la salida
X_train = train_dataset.loc[:, features]
X_test = test_dataset.loc[:, features]
# display(X_train)

# Se construye el dataset de salidas 
y_train = train_dataset.loc[:, salida]
y_test = test_dataset.loc[:, salida]
# display(Y_train)

class_labels = ['Limpio', 'Buggy1', 'Buggy2', 'Buggy3', 'Buggy4', 'Buggy5', 'Buggy6']

X_train.columns = features
X_test.columns = features 

training_data = pd.concat([X_train, y_train], axis = 1)
testing_data = pd.concat([X_test, y_test], axis = 1)
display(training_data)

In [ ]:
#Construccion del modelo
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=0)
# Se entrena el modelo
rf_model.fit(X_train, y_train)  
# print(rf_model)

# LIME

In [ ]:
# Import de Lime 
import lime 
import lime.lime_tabular

# Explicamos el primer metodo del conjunto de test
metodo_explicado = y_test.keys()[0]
metodo_a_explicar = metodo_explicado

# print(f'Explaining {file_to_be_explained} with Lime')

""" 
Construir el modelo explicador de Lime
	training_data: conjunto de entrenamiento con las features
	mode: clasificacion o regresion 
	training_labels: conjunto de entrenamiento con los resultados de las entradas 
	feature_names: lista de nombres de las features analizadas 
	class_names: lista del nombre de las clases ?????
	discretize_continuous: si es True, todas las variables no categóricas seran discretizadas en quartiles.
"""
our_lime_explainer = lime.lime_tabular.LimeTabularExplainer(
	training_data = X_train.values, 
	mode = 'classification', 
	training_labels = y_train,
	feature_names = features, 
	class_names = class_labels, 
	discretize_continuous = True
)

"""
Usar el modelo explicador con la función de predicción para generar la explicación sobre la predicción
	data_row: fila del conjunto de datos que queremos explicar 
	predict_fn: funcion de prediccion 
	num_features: máximo numero de features a las que dar explicación
	top_labels: ignora las labels y produce explicaciones para las K labels con las mejores predicciones de probabilidad donde K es el parametro
"""
lime_local_explanation_of_an_instance = our_lime_explainer.explain_instance(
	data_row = X_test.loc[metodo_explicado, :],
	predict_fn = rf_model.predict_proba,
	num_features = 5,
	top_labels = 1
)

# Visualización de las explicaciones 
lime_local_explanation_of_an_instance.show_in_notebook()
